In [ ]:
import json
from pathlib import Path

import git

repo = git.Repo(Path(".").absolute(), search_parent_directories=True)
ROOT = Path(repo.working_tree_dir)

# Path to the folder where emoji image files are stored
images_path = ROOT / "emoji-images" / "imgs"
# Extract the unicodes from the image file names
image_unicodes = set(p.stem for p in images_path.glob("*.png"))

# Path to the folder where emoji names and unicodes are stored
emojis_path = ROOT / "emojis" / "emojis.json"
# Load the emoji data
with open(emojis_path, "r") as f:
    emojis = json.load(f)["emojis"]

# A dictionary that assigns a name to each unicode with a corresponding image.
unicode2emoji_name = {
    unicode: entry["name"]
    for entry in emojis
    if (unicode := entry["unicode"].replace(" ", "-")) in image_unicodes
    and entry["name"]
}

emoji_name2unicode = {
    unicode2emoji_name[unicode]: unicode for unicode in unicode2emoji_name
}
emoji_names = list(emoji_name2unicode.keys())

def load_emoji(emoji_name, size=(32, 32)):
    """Load the emojis."""
    unicode = emoji_name2unicode[emoji_name]
    image = Image.open(images_path / (unicode + ".png")).convert("RGBA").resize(size)
    return image

In [ ]:
import numpy as np
from munch import Munch
from PIL import Image, ImageDraw, ImageOps
import math

In [ ]:
class Entity:
    def __init__(
        self,
        name=None,
        p=(0, 0),
        theta=0,
        size=(32, 32),
    ):
        self.image = load_emoji(name, size=size)
        self.array = np.array(self.image)[::-1, :]
        # crop the image
        self.mask = self.get_mask()
        bbox = self.get_bbox()
        self.array = self.array[bbox.bottom : bbox.top + 1, bbox.left : bbox.right + 1]
        self.image = Image.fromarray(self.array[::-1, :])
        x_max = self.array.shape[1] - 1
        y_max = self.array.shape[0] - 1
        self.bbox_float = np.array(
            [
                [0, 0],
                [0, y_max],
                [x_max, y_max],
                [x_max, 0],
            ]
        )
        #         self.bbox_float = np.array(
        #             [
        #                 [bbox.left, bbox.bottom],
        #                 [bbox.left, bbox.top],
        #                 [bbox.right, bbox.top],
        #                 [bbox.right, bbox.bottom],
        #             ]
        #         )
        self.center = (self.bbox_float[2] - self.bbox_float[0]) / 2 + self.bbox_float[0]
        self.rotate(theta)
        self.translate(p)
        self.name = name
        self.p = p

    def rotate(self, theta):
        c, s = np.cos(theta), np.sin(theta)
        R = np.array([[c, -s], [s, c]])
        self.bbox_float = (self.bbox_float - self.center) @ R.transpose() + self.center
        self.image = self.image.rotate(
            360 * theta / (2 * math.pi),
            expand=True,
        )
        self.array = np.array(self.image)[::-1, :]
        self.mask = self.get_mask()

    def translate(self, p):
        self.bbox_float += np.array(p)

    def draw(
        self,
        base,
        show_bbox=False,
        bbox_absolute=False,
        bbox_color=None,
        bbox_border=None,
        orientation_marker=False,
    ):
        d = ImageDraw.Draw(base)
        if show_bbox:
            if bbox_absolute:
                bbox = self.get_bbox()
                self.array = self.array[
                    bbox.bottom : bbox.top + 1, bbox.left : bbox.right + 1
                ]
                self.image = Image.fromarray(self.array[::-1, :])
                self.bbox_float = np.array(
                    [
                        [bbox.left, bbox.bottom],
                        [bbox.left, bbox.top],
                        [bbox.right, bbox.top],
                        [bbox.right, bbox.bottom],
                    ]
                )
                self.translate(self.p)
            d.polygon(
                [(p[0], base.size[1] - p[1]) for p in self.bbox],
                fill=bbox_color if bbox_color else None,
                outline=bbox_border if bbox_border else None,
            )
        if orientation_marker:
            # Use the tenth of the bounding box (from the top) for marking the front side of an self.
            bottom_left = ((self.bbox_float[0] - self.bbox_float[1]) / 10).astype(
                int
            ) + self.top_left
            bottom_right = ((self.bbox_float[3] - self.bbox_float[2]) / 10).astype(
                int
            ) + self.top_right
            d.polygon(
                [
                    (p[0], base.size[1] - p[1])
                    for p in (
                        bottom_left,
                        self.top_left,
                        self.top_right,
                        bottom_right,
                    )
                ],
                fill=(255, 255, 255, 255),
            )
        base.alpha_composite(
            self.image,
            (self.bbox[:, 0].min(), canvas.size[1] - self.bbox[:, 1].max()),
        )
        return base

    @property
    def bbox(self):
        """Round the oringinal bbox coordinates."""
        #         self.bbox_float.round().astype(int)
        return np.concatenate(
            (np.round(self.bbox_float[:, :1]), np.round(self.bbox_float[:, 1:2])), axis=1
        ).astype(int)

    @property
    def bottom_left(self):
        return self.bbox[0]

    @property
    def top_left(self):
        return self.bbox[1]

    @property
    def top_right(self):
        return self.bbox[2]

    @property
    def bottom_right(self):
        return self.bbox[3]

    def __repr__(self):
        return f"{self.name}: {', '.join(str(tuple(p)) for p in self.bbox)}"

    def get_mask(self, threshold=None) -> np.array:
        """Return the mask of the image. The mask is obtained from the alpha channel.

        :param image_array: An array of shape (H x W x 4) representing an RGBA image.
        :param threshold: determines the threshold for deciding whether to assign a pixel
            to foreground or background. 4 is a good value. Higher values might not for
            example get the correct mask for the emoji "cigarette" (unicode: 1f6ac).
        :returns: The mask of the input image.
        """
        if threshold is None:
            threshold = 4
        mask_array = self.array[:, :, 3]
        mask_array[mask_array < threshold] = 0  # background
        mask_array[mask_array >= threshold] = 255  # foreground
        return mask_array.astype(bool)

    def get_bbox(self):
        """Get the bounding boxes of the mask_array."""
        x_proj = self.mask.any(axis=0)  # Does the column contain a non-zero entry?
        y_proj = self.mask.any(axis=1)  # Does the row contain a non-zero entry?
        bbox = Munch(
            left=x_proj.argmax(),
            bottom=y_proj.argmax(),
            right=len(x_proj) - 1 - x_proj[::-1].argmax(),
            top=len(y_proj) - 1 - y_proj[::-1].argmax(),
        )
        return bbox

In [ ]:
def draw(entities, canvas):
    """Draw the objects on a canvas."""
    for entity in entities:
        canvas.alpha_composite(
            entity.image,
            (entity.bottom_left[0], canvas.size[1] - entity.top_left[1]),
        )
    return canvas

In [ ]:
entity = Entity(name="octopus", p=(15, 60), theta=0 * 2 * math.pi, size=(64, 64))
canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
entity.draw(canvas, show_bbox=True)
entity = Entity(name="trophy", p=(90, 60), theta=0.5 * 2 * math.pi, size=(64, 64))
entity.draw(canvas, show_bbox=True)

# Test the Class Entity

In [ ]:
from ipywidgets import interact
from qsr_learning.data.bounding_box_data import above, below, left_of, right_of


@interact(
    x1=(0, 150), y1=(0, 150), theta1=(0, 360), x2=(0, 150), y2=(0, 150), theta2=(0, 360)
)
def test_spatial_relations(x1=64, y1=64, theta1=0, x2=128, y2=128, theta2=0):
    entity1 = Entity(
        name="octopus", p=(x1, y1), theta=theta1 / 360 * 2 * math.pi, size=(32, 32)
    )
    canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
    entity1.draw(canvas, show_bbox=True, bbox_absolute=True)
    entity2 = Entity(
        name="trophy", p=(x2, y2), theta=theta2 / 360 * 2 * math.pi, size=(32, 32)
    )
    display(entity2.draw(canvas, show_bbox=True))
    for relation in [left_of, right_of, above, below]:
        if relation(entity1, entity2):
            print(entity1.name, relation.__name__, entity2.name)
    for relation in [left_of, right_of, above, below]:
        if relation(entity2, entity1):
            print(entity2.name, relation.__name__, entity1.name)